In [1]:
import pandas as pd
import numpy as np

In [2]:
## get the vcf file frothe chromosomes and positions 
import allel
## ld pruned vcf file
vcf_file = '/carnegie/nobackup/scratch/xwu/grenet/hapFIRE_updatedVCF/greneNet_final_v1.1.recode.vcf'
vcf = allel.read_vcf(vcf_file)

/home/tbellagio/.local/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
var_pos = pd.DataFrame({'id': vcf['variants/ID'] , 'pos': vcf['variants/POS'], 'chrom': vcf['variants/CHROM']})

In [5]:
var_pos

,id,pos,chrom
0,1_291,291,1
1,1_346,346,1
2,1_349,349,1
3,1_352,352,1
4,1_353,353,1
...,...,...,...
3235475,5_26975208,26975208,5
3235476,5_26975215,26975215,5
3235477,5_26975267,26975267,5
3235478,5_26975272,26975272,5


In [6]:
var_pos.to_csv('../var_pos_grenenet.csv',index=None)

In [9]:
variants = vcf['calldata/GT']

In [ ]:
genotype_counts = np.sum(variants, axis=2)

In [12]:
genotype_counts.shape

(3235480, 231)

In [6]:
vcf.keys()

dict_keys(['samples', 'calldata/GT', 'variants/ALT', 'variants/CHROM', 'variants/FILTER_PASS', 'variants/ID', 'variants/POS', 'variants/QUAL', 'variants/REF'])

In [16]:
labels = vcf['variants/CHROM']

In [17]:
# Dictionary to hold the split arrays
split_arrays = {}

# Populate the dictionary with data arrays split by unique labels
for label in np.unique(labels):
    split_arrays[label] = genotype_counts[labels == label]

# Now `split_arrays` contains each label associated with subarrays of `data`

In [28]:
chrom1 = split_arrays['1']
chrom2 = split_arrays['2']
chrom3 = split_arrays['3']
chrom4 = split_arrays['4']
chrom5 = split_arrays['5']

In [45]:
var_pos = pd.DataFrame({'id': vcf['variants/ID'] , 'pos': vcf['variants/POS']})

In [52]:
def createchr_files(split_arrays, chr_number):
    chrom = split_arrays[chr_number]
    chrom = pd.DataFrame(chrom)
    chr_ids = [i for i in vcf['variants/ID'] if i.startswith(f'{chr_number}_')]
    chrom.index = chr_ids
    var_pos_chr = var_pos[var_pos['id'].isin(chr_ids)]
    chrom.to_csv(f'chr_{chr_number}_geno.csv')
    var_pos_chr.to_csv(f'chr_{chr_number}_varpos.csv',index=None)
    

In [53]:
createchr_files(split_arrays, '1')

In [54]:
for i in ['1', '2', '3', '4', '5']:
    createchr_files(split_arrays, i)

In [44]:
ulimit -s $((10*1024*1024))


In [55]:
10*1024*1024

10485760

In [ ]:
setwd("/carnegie/nobackup/scratch/tbellagio/gea_grene-net/bigld")

library(readr)
library(gpart)
data_g <- read_csv('chr_1_geno.csv')
## trasnform it to set it in the right input shape 
data_g = t(data_g)

#head(data_g)
#dim(data_g)

# Set the first row as column names
colnames(data_g) <- as.character(unlist(data_g[1, ]))

# Remove the first row from the dataframe
data_g <- data_g[-1, ]


SNPinfo_g <- read_csv('chr_1_varpos.csv')
dim(SNPinfo_g)


res <- BigLD(geno = data_g, SNPinfo = SNPinfo_g, CLQmode = "density", CLQcut = 0.6, MAFcut=0.05, hrstType="fast")


In [ ]:
data_g <- read_csv('../leave_1_out/genotype_counts_fullgenome231ecotypes.csv')
## delete the first column which was actually teh index 
data_g <- data_g[, -1]
## trasnform it to set it in the right input shape 
data_g = t(data_g)

#head(data_g)
dim(data_g)


SNPinfo_g <- read_csv('../leave_1_out/grenenet_variants_id_pos.csv')
dim(SNPinfo_g)

##set the columns of the geno matrix to be the variant id 
colnames(data_g) <- SNPinfo_g$variants_id


res <- BigLD(geno = data_g, SNPinfo = SNPinfo_g, CLQmode = "dens

In [ ]:
def CompleteLDPartition(standardized_genotype_matrix,cutoff,window_size):
	
	#define variables
	n_inds,n_snps = standardized_genotype_matrix.shape
	snp_list = {}
	cummax_list = []
	max_list = []
	boundary = []

	alone_SNPs_index = []
	for i in range(n_snps):
		snp_list[i] = find_ld(i,snps=standardized_genotype_matrix,cutoff=0.1,window_size=50)
		if len(snp_list[i]) == 1:
			alone_SNPs_index.append(i)
	if len(alone_SNPs_index) > 0:
		print("QUALITY CHECK: identify %d snps that are not in LD (r2 < %f) with its 50 up/downstream neighbours. You may consider remove these SNPs" %(len(alone_SNPs_index),0.1))

	print("window_size is %d, and the correlation cutoff is %f" %(window_size, cutoff))
	for i in range(n_snps):
		snp_list[i] = find_ld(i,snps=standardized_genotype_matrix,cutoff=cutoff,window_size=window_size)

	for i in range(len(snp_list)):
		if len(snp_list[i]) > 0:
			max_list.append(np.max(snp_list[i]))
		else:
			max_list.append(i)
	
	cummax_list.append(max_list[0])
	for i in range(1,len(max_list)):
		if max_list[i] > cummax_list[i-1]:
			cummax_list.append(max_list[i])
		else:
			cummax_list.append(cummax_list[i-1])

	idx = np.where( cummax_list - np.array(range(n_snps)) == 0)
	idx = idx[0]

	boundary_ = np.concatenate(([-1],np.array(idx)))
	for i in range(len(boundary_)-1):
		left = boundary_[i]+1
		right = boundary_[i+1]
		if right - left > 0:
			boundary.append([left,right])
		else:
			boundary.append([left])

	j = 0

	while j < len(boundary):
		if len(boundary[j]) == 1:
			if j ==0:
				boundary[j+1][0] = boundary[j][0]
				del boundary[j]
			else:
				boundary[j-1][1] = boundary[j][0]
				del boundary[j]
		else:
			j += 1

	return(boundary,alone_SNPs_index)

In [ ]:
with open(args.output+"_fine_genomewide_partition.txt", "w") as GW_BREAKPOITS:
    for ch in chromosome:
        common_fine_breakpoints = bp.BigLD_partition(DIR,IndepLD_common_breakpoints_index[ch],common_geno_matrix[ch],common_variant_names[ch],common_variant_positions[ch],args.CLQcut,args.output)
        gw_fine_breakpoints[ch] = uf.convert_fine_genomewide_breakpoints(common_fine_breakpoints,common_allele_index[ch],len(variant_names[ch]),gw_independent_breakpoints[ch])
        for i in range(len(gw_fine_breakpoints[ch])):
            GW_BREAKPOITS.write(str(ch+"\t"+'\t'.join(map(str, gw_fine_breakpoints[ch][i]))+"\n"))	

In [ ]:
def BigLD_partition(DIR,IndepLD_breakpoints_index,geno_matrix,variant_names,variant_positions,CLQcut,prefix):
	fine_breakpoints_ch = []

	#generate geno and SNPinfo for BigLD
	for I in range(len(IndepLD_breakpoints_index)):
		left = IndepLD_breakpoints_index[I][0]
		right = IndepLD_breakpoints_index[I][1]
		tmp_names = variant_names[left:right+1]
		tmp_positions = variant_positions[left:right+1]
		tmp_matrix = pd.DataFrame(geno_matrix[:,left:right+1],columns= tmp_names)
		tmp_matrix.to_csv(prefix+"_"+str(I)+"_geno_matrix"+".btmp",sep="\t",header=True,index=False)
		INFO = open(prefix+"_"+str(I)+"_snpINFO"+".btmp","w")
		INFO.write("chrN\trsID\tbp\n")
		for j in range(len(tmp_positions)):
			INFO.write(str(1)+"\t"+str(tmp_names[j])+"\t"+str(tmp_positions[j])+"\n")
		INFO.close()

		BigLD_command = "Rscript "+DIR+"/BigLD.R -g "+prefix+"_"+str(I)+"_geno_matrix"+".btmp"+ " -s "+prefix+"_"+str(I)+"_snpINFO"+".btmp" + " -c " + str(CLQcut) + " -o " + prefix+"_"+str(I)
		try:
			blocks = []
			subprocess.check_call(BigLD_command,shell=True)
			tmp_file = prefix+"_"+str(I)+"_res_btmp.txt"
			with open(tmp_file,"r") as INPUT:
				header = INPUT.readline()
				for line in INPUT:
					items = line.split("\t")
					blocks.append([variant_positions.index(int(items[5])),variant_positions.index(int(items[6]))])
			blocks[-1][1] = right

			fine_breakpoints_ch.extend(blocks)

		except subprocess.CalledProcessError as e:
			print("BigLD cannot further partition blocks in this region %i - %i" %(left,right))
			fine_breakpoints_ch.append(IndepLD_breakpoints_index[I])
		rm_command = "rm "+prefix+"_"+str(I)+"_*btmp*"
		subprocess.check_call(rm_command,shell = True)

	return(fine_breakpoints_ch)